In [1]:
import mlflow
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"

### Interacting with the MLflow tracking server

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='file:///c:/BITrusted/fuel-price-prediction/src/mlruns/3', creation_time=1722074481906, experiment_id='3', last_update_time=1722074481906, lifecycle_stage='active', name='client-test-experiment', tags={}>,
 <Experiment: artifact_location='file:///c:/BITrusted/fuel-price-prediction/src/mlruns/2', creation_time=1722000115878, experiment_id='2', last_update_time=1722000115878, lifecycle_stage='active', name='fuel-price-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1721834824177, experiment_id='0', last_update_time=1721834824177, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(experiment_ids='2', order_by=["metrics.avg_val_loss"])

In [6]:
for run in runs:
    print(
        f"run id: {run.info.run_id}, avg_val_loss: {run.data.metrics['avg_val_loss']:.4f}"
    )

run id: f24c78650a074e1aa5b737d86fe784ca, avg_aval_loss: 0.0002


### Interacting with the Model Registry

In [7]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [8]:
run_id = "'337ff4b11daf4118a3c9a64263073c4b'"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="fuel-price-predictor")

Successfully registered model 'fuel-price-predictor'.
Created version '1' of model 'fuel-price-predictor'.


<ModelVersion: aliases=[], creation_timestamp=1722075047822, current_stage='None', description=None, last_updated_timestamp=1722075047822, name='fuel-price-predictor', run_id='f24c78650a074e1aa5b737d86fe784ca', run_link=None, source='file:///c:/BITrusted/fuel-price-prediction/src/mlruns/2/f24c78650a074e1aa5b737d86fe784ca/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [9]:
model_name = "fuel-price-predictor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


C:\Users\emenike\AppData\Local\Temp\ipykernel_21376\1626802265.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name = model_name)


We can see that the model has not been registered and thus has the `stage: None`. We now, register the model.

#### Register the model

In [11]:
# first in staging

model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False,
)

C:\Users\emenike\AppData\Local\Temp\ipykernel_21376\3982449173.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1722075047822, current_stage='Staging', description=None, last_updated_timestamp=1722075264582, name='fuel-price-predictor', run_id='f24c78650a074e1aa5b737d86fe784ca', run_link=None, source='file:///c:/BITrusted/fuel-price-prediction/src/mlruns/2/f24c78650a074e1aa5b737d86fe784ca/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

Now, we add some description to help identify the model

In [14]:
from datetime import datetime


date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}",
)

<ModelVersion: aliases=[], creation_timestamp=1722075047822, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2024-07-27', last_updated_timestamp=1722075434432, name='fuel-price-predictor', run_id='f24c78650a074e1aa5b737d86fe784ca', run_link=None, source='file:///c:/BITrusted/fuel-price-prediction/src/mlruns/2/f24c78650a074e1aa5b737d86fe784ca/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>